In [ ]:
import math

class Node:
    def __init__(self, attribute=None, children=None, leaf=None):
        self.attribute = attribute
        self.children = children if children else {}
        self.leaf = leaf
        
    def is_leaf(self):
        return self.leaf is not None

def load_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            row = list(map(float, line.split()))
            data.append(row)
    return data

def get_majority_class(data):
    class_count = {}
    for row in data:
        label = int(row[-1])
        if label not in class_count:
            class_count[label] = 0
        class_count[label] += 1
    majority_class = max(class_count, key=class_count.get)
    return majority_class

def get_entropy(data):
    class_count = {}
    for row in data:
        label = int(row[-1])
        if label not in class_count:
            class_count[label] = 0
        class_count[label] += 1
    entropy = 0
    for count in class_count.values():
        prob = count / len(data)
        entropy += -prob * math.log(prob, 2)
    return entropy

def get_attribute_values(data, attribute):
    values = set()
    for row in data:
        values.add(row[attribute])
    return sorted(list(values))

def get_attribute_with_max_gain(data, used_attributes):
    entropy_s = get_entropy(data)
    max_gain = 0
    max_attribute = None
    for attribute in range(len(data[0]) - 1):
        if attribute in used_attributes:
            continue
        attribute_values = get_attribute_values(data, attribute)
        attribute_entropy = 0
        for value in attribute_values:
            subset = [row for row in data if row[attribute] == value]
            prob = len(subset) / len(data)
            attribute_entropy += prob * get_entropy(subset)
        gain = entropy_s - attribute_entropy
        if gain > max_gain:
            max_gain = gain
            max_attribute = attribute
    return max_attribute

def id3(data, used_attributes):
    # If all instances belong to the same class, return a leaf node with that class
    labels = set(row[-1] for row in data)
    if len(labels) == 1:
        return Node(leaf=labels.pop())
    
    # If there are no more attributes to split on, return a leaf node with the majority class
    if len(used_attributes) == len(data[0]) - 1:
        majority_class = get_majority_class(data)
        return Node(leaf=majority_class)
    
    # Choose the attribute with the highest information gain
    attribute = get_attribute_with_max_gain(data, used_attributes)
    node = Node(attribute)
    used_attributes.add(attribute)
    
    # Recurse on each value of the chosen attribute
    attribute_values = get_attribute_values(data, attribute)
    for value in attribute_values:
        subset = [row for row in data if row[attribute] == value]
        if len(subset) == 0:
            # If there are no instances with this attribute value, return a leaf node with the majority class
            majority_class = get_majority_class(data)
            node.children[value] = Node(leaf=majority_class)
        else:
            node.children[value] = id3(subset, used_attributes.copy())
    used_attributes.remove(attribute)
    return node


In [ ]:
# Load training and test data
train_data = load_data('/content/pp_tra.dat')
test_data = load_data('/content/pp_tes.dat')

# Build decision tree using ID3 algorithm
root = id3(train_data, set())

# Predict class labels for test data using decision tree
correct_count = 0
for row in test_data:
    node = root
    while not node.is_leaf():
        attribute = node.attribute
        value = row[attribute]
        if value not in node.children:
            # If a value is encountered that was not in the training data, use the majority class at that node
            majority_class = get_majority_class(train_data)
            node = Node(leaf=majority_class)
        else:
            node = node.children[value]
    if node.leaf == row[-1]:
        correct_count += 1

# Compute accuracy of predictions
accuracy = correct_count / len(test_data)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6024602460246025
